In [1]:
import json
import numpy as np
import pandas as pd
import re

In [5]:
df = pd.read_csv("san_pham31.csv")  # Nếu bạn đọc từ file CSV

In [6]:
# Chuyển 0 thành np.nan
df['total_count'] = df['total_count'].replace(0, np.nan)



In [10]:
df.isna().sum()

product_id                   0
name                         0
price                        0
special_price                0
total_count                780
manufacturer                 0
operating_system             0
os_version                   0
battery                      0
chipset                      0
display_size                 0
display_resolution           0
mobile_type_of_display       0
memory_internal              0
storage                      0
product_weight               0
mobile_tan_so_quet           0
loai_mang                    0
mobile_cong_sac              0
mobile_cam_bien_van_tay      0
warranty_information         0
average_rating               0
key_selling_points           0
promotion_information        0
dtype: int64

In [8]:
# Tính trung vị
median_value = df['total_count'].median()
median_value

np.float64(3.0)

In [9]:
max = df['total_count'].max()
max

np.float64(204.0)

In [11]:
min = df['total_count'].min()
min

np.float64(1.0)

In [12]:
mean=df['total_count'].mean()
mean

np.float64(12.544747081712062)

In [35]:
# 1. Tạo bảng Manufacturers
manufacturers_df = df[['manufacturer']].drop_duplicates().reset_index(drop=True)
manufacturers_df.columns = ['manufacturer_name']
manufacturers_df['manufacturer_id'] = manufacturers_df.index + 1  # Gán id từ 1 trở đi

# 2. Tạo bảng Operating Systems
operating_systems_df = df[['operating_system', 'os_version']].drop_duplicates().reset_index(drop=True)
operating_systems_df.columns = ['os_name', 'os_version']
operating_systems_df['os_id'] = operating_systems_df.index + 1001  # Gán id từ 1001 trở đi

# 3. Tạo bảng Display
display_df = df[['display_size', 'display_resolution', 'mobile_type_of_display']].drop_duplicates().reset_index(drop=True)
display_df.columns = ['display_size', 'display_resolution', 'mobile_type_of_display']
display_df['display_id'] = display_df.index + 2001  # Gán id từ 2001 trở đi

# 4. Tạo bảng Products và giữ lại manufacturer_id và os_id, cũng như các cột cần thiết cho phép nối với display_df
products_df = df[['product_id', 'name', 'price', 'special_price', 
                   'total_count', 'manufacturer', 
                   'operating_system', 'battery', 'chipset', 
                   'memory_internal', 'storage', 
                   'product_weight', 
                   'mobile_tan_so_quet', 'loai_mang', 
                   'mobile_cong_sac', 'mobile_cam_bien_van_tay', 
                   'warranty_information', 'average_rating', 
                   'key_selling_points', 'promotion_information', 
                   'display_size', 'display_resolution']].copy()  # Thêm display_size và display_resolution vào đây

# Ghép manufacturer_id vào products_df
products_df = products_df.merge(manufacturers_df, 
                                 how='left', 
                                 left_on='manufacturer', 
                                 right_on='manufacturer_name')

# Ghép os_id vào products_df
products_df = products_df.merge(operating_systems_df[['os_name', 'os_version', 'os_id']], 
                                 how='left', 
                                 left_on='operating_system', 
                                 right_on='os_name')

# Ghép display_id vào products_df
products_df = products_df.merge(display_df, 
                                 how='left', 
                                 left_on=['display_size', 'display_resolution'], 
                                 right_on=['display_size', 'display_resolution'])

# Xóa cột không cần thiết
products_df = products_df[['product_id', 'name', 'price', 'special_price', 
                             'total_count', 'manufacturer_id', 'os_id', 
                             'display_id', 'battery', 'chipset', 
                             'memory_internal', 'storage', 
                             'product_weight', 
                             'mobile_tan_so_quet', 'loai_mang', 
                             'mobile_cong_sac', 'mobile_cam_bien_van_tay', 
                             'warranty_information', 'average_rating', 
                             'key_selling_points', 'promotion_information']]

# Xuất ra file CSV
manufacturers_df.to_csv('manufacturers.csv', index=False, encoding='utf-8-sig')
operating_systems_df.to_csv('operating_systems.csv', index=False, encoding='utf-8-sig')
display_df.to_csv('display.csv', index=False, encoding='utf-8-sig')
products_df.to_csv('products.csv', index=False, encoding='utf-8-sig')


print("Chovy")

Chovy


In [39]:
manufacturers_df.dtypes


manufacturer_name    object
manufacturer_id       int64
dtype: object

In [40]:
operating_systems_df.dtypes

os_name       object
os_version    object
os_id          int64
dtype: object

In [41]:
display_df.dtypes

display_size              float64
display_resolution         object
mobile_type_of_display     object
display_id                  int64
dtype: object

In [42]:
products_df.dtypes

product_id                   int64
name                        object
price                      float64
special_price              float64
total_count                  int64
manufacturer_id              int64
os_id                        int64
display_id                   int64
battery                      int64
chipset                     object
memory_internal            float64
storage                    float64
product_weight             float64
mobile_tan_so_quet         float64
loai_mang                  float64
mobile_cong_sac             object
mobile_cam_bien_van_tay     object
warranty_information        object
average_rating             float64
key_selling_points          object
promotion_information       object
dtype: object

In [ ]:
import pandas as pd
from pymongo import MongoClient
import psycopg2

# Kết nối đến MongoDB
mongo_client = MongoClient('mongodb://localhost:27017/')
mongo_db = mongo_client['ptdl_data']  # Tên database của bạn

# Các collection mà bạn đã tạo
collections = {
    'telephone': mongo_db['telephone'],
    'manu': mongo_db['manu'],
    'os': mongo_db['os'],
    'display': mongo_db['display'],
    'products': mongo_db['products']
}

# Kết nối đến PostgreSQL
postgres_connection = psycopg2.connect(
    database="your_database",       # Tên cơ sở dữ liệu PostgreSQL
    user="your_username",           # Tên người dùng
    password="your_password",       # Mật khẩu
    host="localhost",
    port="5432"
)
postgres_cursor = postgres_connection.cursor()

# Chèn dữ liệu vào PostgreSQL
def insert_data_to_postgres(collection_name, data):
    if collection_name == 'manu':
        for row in data:
            insert_query = "INSERT INTO manu (manufacturer_id, manufacturer_name) VALUES (%s, %s);"
            postgres_cursor.execute(insert_query, (row['manufacturer_id'], row['manufacturer_name']))
    
    elif collection_name == 'os':
        for row in data:
            insert_query = "INSERT INTO os (os_id, os_name, os_version) VALUES (%s, %s, %s);"
            postgres_cursor.execute(insert_query, (row['os_id'], row['os_name'], row['os_version']))
    
    elif collection_name == 'display':
        for row in data:
            insert_query = "INSERT INTO display (display_id, display_size, display_resolution, mobile_type_of_display) VALUES (%s, %s, %s, %s);"
            postgres_cursor.execute(insert_query, (row['display_id'], row['display_size'], row['display_resolution'], row['mobile_type_of_display']))
    
    elif collection_name == 'products':
        for row in data:
            insert_query = "INSERT INTO products (product_id, name, price, special_price, total_count, manufacturer_id, os_id, display_id, battery, chipset) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            postgres_cursor.execute(insert_query, (row['product_id'], row['name'], row['price'], row['special_price'], row['total_count'], row['manufacturer_id'], row['os_id'], row['display_id'], row['battery'], row['chipset']))

    elif collection_name == 'telephone':
        for row in data:
            insert_query = "INSERT INTO telephone (product_id, name, price, special_price, total_count, manufacturer_id, os_id, display_id, battery, chipset) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            postgres_cursor.execute(insert_query, (row['product_id'], row['name'], row['price'], row['special_price'], row['total_count'], row['manufacturer_id'], row['os_id'], row['display_id'], row['battery'], row['chipset']))

# Lặp qua các collection và chèn dữ liệu vào PostgreSQL
for name, collection in collections.items():
    data = list(collection.find())  # Lấy tất cả dữ liệu từ collection
    insert_data_to_postgres(name, data)

# Lưu thay đổi vào PostgreSQL
postgres_connection.commit()

# Đóng kết nối
postgres_cursor.close()
postgres_connection.close()
mongo_client.close()

print("All data has been inserted into PostgreSQL successfully.")

In [43]:
df.dtypes

product_id                   int64
name                        object
price                      float64
special_price              float64
total_count                  int64
manufacturer                object
operating_system            object
os_version                  object
battery                      int64
chipset                     object
display_size               float64
display_resolution          object
mobile_type_of_display      object
memory_internal            float64
storage                    float64
product_weight             float64
mobile_tan_so_quet         float64
loai_mang                  float64
mobile_cong_sac             object
mobile_cam_bien_van_tay     object
warranty_information        object
average_rating             float64
key_selling_points          object
promotion_information       object
dtype: object

In [ ]:
CREATE TABLE telephone (
    product_id INT PRIMARY KEY,              
    name TEXT,                              
    price DOUBLE PRECISION,                 
    special_price DOUBLE PRECISION,         
    total_count INT,                        
    manufacturer TEXT,                      
    operating_system TEXT,                  
    os_version TEXT,                        
    battery INT,                             
    chipset TEXT,                           
    display_size DOUBLE PRECISION,           
    display_resolution TEXT,                 
    mobile_type_of_display TEXT,            
    memory_internal DOUBLE PRECISION,        
    storage DOUBLE PRECISION,                
    product_weight DOUBLE PRECISION,         
    mobile_tan_so_quet DOUBLE PRECISION,    
    loai_mang DOUBLE PRECISION,              
    mobile_cong_sac TEXT,                   
    mobile_cam_bien_van_tay TEXT,           
    warranty_information TEXT,               
    average_rating DOUBLE PRECISION,        
    key_selling_points TEXT,                
    promotion_information TEXT               
);

In [ ]:
CREATE TABLE manu (
    manufacturer_id SERIAL PRIMARY KEY,
    manufacturer_name VARCHAR(255) NOT NULL
);

CREATE TABLE os (
    os_id SERIAL PRIMARY KEY,
    os_name VARCHAR(255) NOT NULL,
    os_version VARCHAR(255) NOT NULL
);

CREATE TABLE display (
    display_id SERIAL PRIMARY KEY,
    display_size VARCHAR(255) NOT NULL,
    display_resolution VARCHAR(255) NOT NULL,
    mobile_type_of_display VARCHAR(255) NOT NULL
);

CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    price DECIMAL(15, 2) NOT NULL,
    special_price DECIMAL(15, 2),
    total_count INT NOT NULL,
    manufacturer_id INT REFERENCES Manufacturers(manufacturer_id),
    os_id INT REFERENCES Operating_Systems(os_id),
    display_id INT REFERENCES Display(display_id),
    battery VARCHAR(50),
    chipset VARCHAR(100),
    memory_internal VARCHAR(255),
    storage VARCHAR(100),
    product_weight VARCHAR(50),
    mobile_tan_so_quet VARCHAR(50),
    loai_mang VARCHAR(50),
    mobile_cong_sac VARCHAR(50),
    mobile_cam_bien_van_tay VARCHAR(50),
    warranty_information TEXT,
    average_rating DECIMAL(3, 2),
    key_selling_points TEXT,
    promotion_information TEXT
);

CREATE TABLE telephone (
    product_id INT PRIMARY KEY,              
    name TEXT,                              
    price DOUBLE PRECISION,                 
    special_price DOUBLE PRECISION,         
    total_count INT,                        
    manufacturer TEXT,                      
    operating_system TEXT,                  
    os_version TEXT,                        
    battery INT,                             
    chipset TEXT,                           
    display_size DOUBLE PRECISION,           
    display_resolution TEXT,                 
    mobile_type_of_display TEXT,            
    memory_internal DOUBLE PRECISION,        
    storage DOUBLE PRECISION,                
    product_weight DOUBLE PRECISION,         
    mobile_tan_so_quet DOUBLE PRECISION,    
    loai_mang DOUBLE PRECISION,              
    mobile_cong_sac TEXT,                   
    mobile_cam_bien_van_tay TEXT,           
    warranty_information TEXT,               
    average_rating DOUBLE PRECISION,        
    key_selling_points TEXT,                
    promotion_information TEXT               
);

In [ ]:
CREATE TABLE Products (
    product_id SERIAL PRIMARY KEY,  
    name VARCHAR(255) NOT NULL,  
    price NUMERIC(10, 2) NOT NULL,  
    special_price NUMERIC(10, 2),  
    total_count INT NOT NULL,  
    manufacturer_id INT NOT NULL,  
    os_id INT NOT NULL, 
    display_id INT,  
    battery VARCHAR(100),  
    chipset VARCHAR(100),  
    memory_internal VARCHAR(50),  
    storage VARCHAR(50),  
    product_weight VARCHAR(50), 
    mobile_tan_so_quet VARCHAR(50),  
    loai_mang VARCHAR(50),  
    mobile_cong_sac VARCHAR(50),  
    mobile_cam_bien_van_tay VARCHAR(50),  
    warranty_information VARCHAR(100), 
    average_rating NUMERIC(2, 1),  
    key_selling_points TEXT,  
    promotion_information TEXT,  
    FOREIGN KEY (manufacturer_id) REFERENCES Manufacturers(manufacturer_id),  
    FOREIGN KEY (os_id) REFERENCES Operating_Systems(os_id),  
    FOREIGN KEY (display_id) REFERENCES Display(display_id)  
);

In [ ]:
from psycopg2.extras import RealDictCursor
import pyodbc
import psycopg2

# Kết nối tới PostgreSQL
postgres_connection = psycopg2.connect(
    dbname='mydatabase',  # Thay đổi theo tên cơ sở dữ liệu của bạn
    user='postgres',      # Thay đổi theo tên người dùng của bạn
    password='141216',    # Thay đổi theo mật khẩu của bạn
    host='localhost',     # Thay đổi nếu cần
    port='5432'           # Thay đổi nếu cần
)
postgres_cursor = postgres_connection.cursor(cursor_factory=RealDictCursor)

# Kết nối tới SQL Server
sqlserver_connection = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=KID1412\SQLEXPRESS;'  # Thay đổi theo máy chủ của bạn
    'DATABASE=PTDL_CK;'            # Thay đổi theo tên cơ sở dữ liệu của bạn
    'Trusted_Connection=yes;'
)
sqlserver_cursor = sqlserver_connection.cursor()

try:
    # Lấy dữ liệu từ bảng telephone trong PostgreSQL
    postgres_cursor.execute("SELECT product_id, name, price, special_price, total_count, manufacturer, operating_system, os_version, battery, chipset, display_size, display_resolution, mobile_type_of_display, memory_internal, storage, product_weight, mobile_tan_so_quet, loai_mang, mobile_cong_sac, mobile_cam_bien_van_tay, warranty_information, average_rating, key_selling_points, promotion_information FROM telephone")
    rows = postgres_cursor.fetchall()

    # Kiểm tra dữ liệu
    print("Dữ liệu lấy từ PostgreSQL:", rows)

    # Bật IDENTITY_INSERT cho bảng telephone trong SQL Server (nếu cần thiết)
    sqlserver_cursor.execute("SET IDENTITY_INSERT telephone ON")

    try:
        # Chèn dữ liệu vào SQL Server
        insert_query = "INSERT INTO telephone (product_id, name, price, special_price, total_count, manufacturer, operating_system, os_version, battery, chipset, display_size, display_resolution, mobile_type_of_display, memory_internal, storage, product_weight, mobile_tan_so_quet, loai_mang, mobile_cong_sac, mobile_cam_bien_van_tay, warranty_information, average_rating, key_selling_points, promotion_information) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
        for row in rows:
            print(f"Chèn dữ liệu: {row}")  # In từng row để kiểm tra

            # Chèn dữ liệu vào SQL Server với cả product_id
            sqlserver_cursor.execute(insert_query, (
                row['product_id'],
                row['name'],
                row['price'],
                row['special_price'],
                row['total_count'],
                row['manufacturer'],
                row['operating_system'],
                row['os_version'],
                row['battery'],
                row['chipset'],
                row['display_size'],
                row['display_resolution'],
                row['mobile_type_of_display'],
                row['memory_internal'],
                row['storage'],
                row['product_weight'],
                row['mobile_tan_so_quet'],
                row['loai_mang'],
                row['mobile_cong_sac'],
                row['mobile_cam_bien_van_tay'],
                row['warranty_information'],
                row['average_rating'],
                row['key_selling_points'],
                row['promotion_information']
            ))

    except Exception as insert_error:
        print(f"Lỗi khi chèn dữ liệu: {insert_error}")
        sqlserver_connection.rollback()  # Rollback nếu có lỗi xảy ra

    finally:
        # Tắt IDENTITY_INSERT sau khi hoàn tất
        sqlserver_cursor.execute("SET IDENTITY_INSERT telephone OFF")

    # Thực hiện commit
    sqlserver_connection.commit()
    print("Dữ liệu đã được chuyển từ PostgreSQL sang SQL Server thành công!")

except Exception as general_error:
    print(f"Lỗi xảy ra: {general_error}")

finally:
    # Đóng kết nối
    postgres_cursor.close()
    postgres_connection.close()
    sqlserver_cursor.close()
    sqlserver_connection.close()

In [ ]:
# 1. Tạo bảng Manufacturers
manufacturers_df = df[['manufacturer']].drop_duplicates().reset_index(drop=True)
manufacturers_df.columns = ['manufacturer_name']
manufacturers_df['manufacturer_id'] = manufacturers_df.index + 1  # Gán id từ 1 trở đi

# 2. Tạo bảng Operating Systems
operating_systems_df = df[['operating_system', 'os_version']].drop_duplicates().reset_index(drop=True)
operating_systems_df.columns = ['os_name', 'os_version']
operating_systems_df['os_id'] = operating_systems_df.index + 1001  # Gán id từ 1001 trở đi

# 3. Tạo bảng Display
display_df = df[['display_size', 'display_resolution', 'mobile_type_of_display']].drop_duplicates().reset_index(drop=True)
display_df.columns = ['display_size', 'display_resolution', 'mobile_type_of_display']
display_df['display_id'] = display_df.index + 2001  # Gán id từ 2001 trở đi

# 4. Tạo bảng Products và giữ lại manufacturer_id và os_id, cũng như các cột cần thiết cho phép nối với display_df
products_df = df[['product_id', 'name', 'price', 'special_price', 
                   'total_count', 'manufacturer', 
                   'operating_system', 'battery', 'chipset', 
                   'memory_internal', 'storage', 
                   'product_weight', 
                   'mobile_tan_so_quet', 'loai_mang', 
                   'mobile_cong_sac', 'mobile_cam_bien_van_tay', 
                   'warranty_information', 'average_rating', 
                   'key_selling_points', 'promotion_information', 
                   'display_size', 'display_resolution']].copy()  # Thêm display_size và display_resolution vào đây

# Ghép manufacturer_id vào products_df
products_df = products_df.merge(manufacturers_df, 
                                 how='left', 
                                 left_on='manufacturer', 
                                 right_on='manufacturer_name')

# Ghép os_id vào products_df
products_df = products_df.merge(operating_systems_df[['os_name', 'os_version', 'os_id']], 
                                 how='left', 
                                 left_on='operating_system', 
                                 right_on='os_name')

# Ghép display_id vào products_df
products_df = products_df.merge(display_df, 
                                 how='left', 
                                 left_on=['display_size', 'display_resolution'], 
                                 right_on=['display_size', 'display_resolution'])

# Xóa cột không cần thiết
products_df = products_df[['product_id', 'name', 'price', 'special_price', 
                             'total_count', 'manufacturer_id', 'os_id', 
                             'display_id', 'battery', 'chipset', 
                             'memory_internal', 'storage', 
                             'product_weight', 
                             'mobile_tan_so_quet', 'loai_mang', 
                             'mobile_cong_sac', 'mobile_cam_bien_van_tay', 
                             'warranty_information', 'average_rating', 
                             'key_selling_points', 'promotion_information']]

# Xuất ra file CSV
manufacturers_df.to_csv('manufacturers.csv', index=False, encoding='utf-8-sig')
operating_systems_df.to_csv('operating_systems.csv', index=False, encoding='utf-8-sig')
display_df.to_csv('display.csv', index=False, encoding='utf-8-sig')
products_df.to_csv('products.csv', index=False, encoding='utf-8-sig')


print("Chovy")

In [ ]:
import pandas as pd
from pymongo import MongoClient


# Tạo danh sách chứa tất cả DataFrame
dataframes = [df, manufacturers_df, operating_systems_df, display_df, products_df]
# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Địa chỉ MongoDB
db = client['ptdl_cleandata']  # Thay 'your_database_name' bằng tên database của bạn

# Tên collection cho từng DataFrame nếu cần
collection_names = ['telephone', 'manu', 'os', 'display', 'products']

# Lặp qua các DataFrame và chèn vào MongoDB
for df, collection_name in zip(dataframes, collection_names):
    collection = db[collection_name]  # Chọn collection tương ứng
    data_to_insert = df.to_dict(orient='records')  # Chuyển đổi DataFrame thành danh sách dict
    collection.insert_many(data_to_insert)  # Chèn dữ liệu vào MongoDB

print("All dataframes have been inserted into MongoDB successfully.")

In [ ]:
CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    price DECIMAL(15, 2) NOT NULL,
    special_price DECIMAL(15, 2),
    total_count INT NOT NULL,
    manufacturer_id INT REFERENCES Manufacturers(manufacturer_id),
    os_id INT REFERENCES Operating_Systems(os_id),
    display_id INT REFERENCES Display(display_id),
    battery VARCHAR(50),
    chipset VARCHAR(100),
    memory_internal VARCHAR(255),
    storage VARCHAR(100),
    product_weight VARCHAR(50),
    mobile_tan_so_quet VARCHAR(50),
    loai_mang VARCHAR(50),
    mobile_cong_sac VARCHAR(50),
    mobile_cam_bien_van_tay VARCHAR(50),
    warranty_information TEXT,
    average_rating DECIMAL(3, 2),
    key_selling_points TEXT,
    promotion_information TEXT
);